In [1]:
import numpy as np 
import pandas as pd 

In [2]:
df_train = pd.read_csv('./data/apply_train.csv')

In [3]:
df_train.head()

,resume_seq,recruitment_seq
0,U05833,R03838
1,U06456,R02144
2,U07807,R01877
3,U04842,R02463
4,U08336,R00112


In [4]:
resume_pool = set(df_train['resume_seq'].unique())
recruitment_pool = set(df_train['recruitment_seq'].unique())

len(resume_pool), len(recruitment_pool)

(8482, 6695)

In [6]:
import random

# 신청한 회사
df_add_negative = df_train.groupby(['resume_seq'])['recruitment_seq'].apply(set).reset_index().rename(columns={'recruitment_seq':'interacted_iid'})  
# 아직 신청하지 않은 회사 
df_add_negative['negative_iid'] = df_add_negative['interacted_iid'].map(lambda x: recruitment_pool - x) 
# 신청한 수
df_add_negative['interacted_iid_cnt'] = df_add_negative['interacted_iid'].map(lambda x: len(x))  
# 신청한 횟수만큼 신청하지 않은 회사 추출..
df_add_negative['negative_sampling'] = df_add_negative.apply(lambda row: random.sample(list(row['negative_iid']), row['interacted_iid_cnt']), axis=1) 
# 검증 코드 
df_add_negative['negative_sampling_cnt'] = df_add_negative['negative_sampling'].map(lambda x: len(x))
df_add_negative['is_error'] = df_add_negative.apply(lambda row: False if row['interacted_iid_cnt'] == row['negative_sampling_cnt'] else True, axis=1)
df_add_negative['is_error'].sum()


0

In [7]:
df_add_negative.head()

,resume_seq,interacted_iid,negative_iid,interacted_iid_cnt,negative_sampling,negative_sampling_cnt,is_error
0,U00001,"{R05288, R04536, R05210, R06065}","{R01115, R04462, R04469, R03770, R06266, R0302...",4,"[R00542, R05035, R04114, R04944]",4,False
1,U00002,"{R01960, R04588, R02946, R01103, R06019, R0234...","{R01115, R04462, R04469, R03770, R06266, R0302...",8,"[R02434, R05161, R04951, R06517, R04494, R0595...",8,False
2,U00003,"{R04918, R01460, R03301}","{R01115, R04462, R04469, R03770, R06266, R0302...",3,"[R03813, R01882, R01573]",3,False
3,U00004,"{R04166, R01747, R02003, R01825, R04891, R0031...","{R01115, R04462, R04469, R03770, R06266, R0302...",17,"[R02514, R03893, R05954, R04241, R06158, R0329...",17,False
4,U00005,"{R03914, R00374, R00832}","{R01115, R04462, R04469, R03770, R06266, R0302...",3,"[R05556, R06415, R01395]",3,False


In [8]:
df_add_negative[['resume_seq','interacted_iid', 'negative_sampling']].head()

,resume_seq,interacted_iid,negative_sampling
0,U00001,"{R05288, R04536, R05210, R06065}","[R00542, R05035, R04114, R04944]"
1,U00002,"{R01960, R04588, R02946, R01103, R06019, R0234...","[R02434, R05161, R04951, R06517, R04494, R0595..."
2,U00003,"{R04918, R01460, R03301}","[R03813, R01882, R01573]"
3,U00004,"{R04166, R01747, R02003, R01825, R04891, R0031...","[R02514, R03893, R05954, R04241, R06158, R0329..."
4,U00005,"{R03914, R00374, R00832}","[R05556, R06415, R01395]"


In [9]:
df_interacted = df_add_negative[['resume_seq', 'interacted_iid']].explode('interacted_iid').rename(
        columns={'interacted_iid':'recruitment_seq'}
    )
df_interacted['target'] = 1
df_interacted.reset_index(drop=True, inplace=True)

print(df_interacted.shape)
df_interacted.head()

(57946, 3)


,resume_seq,recruitment_seq,target
0,U00001,R05288,1
1,U00001,R04536,1
2,U00001,R05210,1
3,U00001,R06065,1
4,U00002,R01960,1


In [10]:
df_negatived = df_add_negative[['resume_seq', 'negative_sampling']].explode('negative_sampling').rename(
        columns={'negative_sampling':'recruitment_seq'}
    )
df_negatived['target'] = 0
df_negatived.reset_index(drop=True, inplace=True)

print(df_negatived.shape)
df_negatived.head()

(57946, 3)


,resume_seq,recruitment_seq,target
0,U00001,R00542,0
1,U00001,R05035,0
2,U00001,R04114,0
3,U00001,R04944,0
4,U00002,R02434,0


In [11]:
df_concat = pd.concat([df_interacted, df_negatived], axis=0)

print(df_concat.shape)
df_concat.head()

(115892, 3)


,resume_seq,recruitment_seq,target
0,U00001,R05288,1
1,U00001,R04536,1
2,U00001,R05210,1
3,U00001,R06065,1
4,U00002,R01960,1


In [12]:
df_shuffle = df_concat.sample(frac=1).reset_index(drop=True)

print(df_shuffle.shape)
df_shuffle.head()

(115892, 3)


,resume_seq,recruitment_seq,target
0,U07023,R00863,1
1,U03442,R02686,0
2,U03708,R04387,0
3,U02065,R05880,0
4,U01147,R05959,0


In [13]:
df_shuffle['target'].value_counts()

target
1    57946
0    57946
Name: count, dtype: int64

In [14]:
df_shuffle['recruitment_seq'].value_counts()

recruitment_seq
R03237    89
R01214    78
R00944    76
R00056    72
R00773    70
          ..
R04475     5
R04734     5
R05176     5
R01029     5
R04611     4
Name: count, Length: 6695, dtype: int64

In [15]:
df_shuffle['recruitment_seq'].describe()

count     115892
unique      6695
top       R03237
freq          89
Name: recruitment_seq, dtype: object